In [47]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [48]:
device = "cuda:0"

In [49]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [50]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [51]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


In [52]:
label = torch.zeros_like(logits)
label[:,2] = 1
loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, label)

In [53]:
print(loss.grad)
# 该警告信息提示你在访问一个非叶子张量（non-leaf Tensor）的 .grad 属性时，可能会遇到问题
# 叶子张量: 在 PyTorch 中，叶子张量是指那些直接由用户创建并且没有经过任何操作生成的张量。这些张量的 .grad 属性会在调用 backward() 后自动填充。
# 非叶子张量: 这些张量是通过对叶子张量进行操作生成的。它们的 .grad 属性不会自动填充，除非你在创建它们时调用了 .retain_grad()。
loss.retain_grad()
print(loss.grad)

None
None


/tmp/ipykernel_68168/4070226966.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(loss.grad)


In [54]:
print(loss.grad_fn) # 是由哪个操作生成的


In [55]:
# AddmmBackward0: w@X + b 操作
# AccumulateGrad: 梯度累计
    # 非叶子张量的反向传播
        # 当你对一个非叶子张量调用 backward() 方法时，AccumulateGrad 会出现。非叶子张量是指那些通过其他张量的操作生成的张量，而不是直接由用户创建的张量。
        # 例如，如果你有一个张量 y 是通过对叶子张量 x 进行某种操作（如加法、乘法等）生成的，那么在调用 y.backward() 时，AccumulateGrad 会被用来将计算得到的梯度累加到 x 的 .grad 属性中。
    # 多次调用 backward()
        # 在反向传播过程中，如果你多次调用 backward()，AccumulateGrad 会将每次反向传播计算得到的梯度累加到相应的张量中。这在训练过程中非常常见，因为你可能会在多个步骤中计算损失并更新梯度
    # 梯度累积
        # 在一些特定的训练策略中，例如梯度累积（gradient accumulation），你可能会在多个小批次上计算梯度，而不是在每个批次上立即更新参数。这种情况下，使用 AccumulateGrad 可以帮助你在每次调用 backward() 时将梯度累加到参数上，直到达到一个指定的更新频率
    
print(loss.grad_fn.next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions[0][0].next_functions)


((<AddmmBackward0 object at 0x7f4a16ec2200>, 0), (None, 0))
((<AccumulateGrad object at 0x7f4a16ec3070>, 0), (<ReluBackward0 object at 0x7f4a16ec2b90>, 0), (<TBackward0 object at 0x7f4a16ec0700>, 0))
()
((<AddmmBackward0 object at 0x7f4a16ec3070>, 0),)
((<AccumulateGrad object at 0x7f4a16ec2410>, 0), (<ReluBackward0 object at 0x7f4a16ec25c0>, 0), (<TBackward0 object at 0x7f4a16ec2b90>, 0))


In [56]:
loss.backward()

In [57]:
w = next(model.parameters())
print(w.shape)
print(w.grad)

torch.Size([512, 784])
tensor([[-6.3574e-04, -1.8195e-04, -5.9267e-04,  ..., -2.2853e-04,
         -3.9521e-05, -4.0951e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.5672e-03,  4.4854e-04,  1.4610e-03,  ...,  5.6337e-04,
          9.7424e-05,  1.0095e-03],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.8031e-04, -5.1605e-05, -1.6809e-04,  ..., -6.4816e-05,
         -1.1209e-05, -1.1614e-04]], device='cuda:0')


In [58]:
# 取消梯度
# w.required_grad = False
# detach
w_det = w.detach()
# w_det和w共享内存，只是w_det没有梯度


In [59]:
# 计算图：DAG
# 在 PyTorch 中，DAG 是动态的。每次调用 backward() 方法后，计算图会被重新创建。这意味着每次反向传播时，都会从头开始构建一个新的计算图
# 由于图的动态性，可以在模型的任何迭代中更改计算图的形状、大小和所执行的操作。这意味着用户可以在模型中使用条件语句、循环等控制流结构，灵活地改变计算过

# 自动微分机制：https://pytorch.org/docs/stable/notes/autograd.html
# Functions母函数: https://pytorch.org/docs/stable/generated/torch.autograd.Function.backward.html#torch.autograd.Function.backward
# 损失函数：https://pytorch.org/docs/stable/nn.html#loss-functions
# 优化算法：https://pytorch.org/docs/stable/optim.html

In [76]:
torch.save(model,"model.pth")
a = torch.load("model.pth",weights_only=False)
torch.save(model.state_dict(),"model.pth")
a = torch.load("model.pth",weights_only=True)
a = torch.load("model.pth",weights_only=False)
# 加载大模型：mmap 参数允许 PyTorch 在加载大型检查点时，使用内存映射（memory-mapped）文件的技术。这意味着文件的内容不会被完全加载到物理内存中，而是映射到虚拟内存中，操作系统仅在需要时才会加载具体的数据块。
a = torch.load("model.pth",mmap=True,weights_only=True)
a = torch.load("model.pth",mmap=True,map_location="cpu",weights_only=True)


In [90]:
# 使用 TorchScript 格式，您将能够加载导出的模型并运行推理，而无需定义模型类。
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_scripted.pt')
model = torch.jit.load('model_scripted.pt')
model.eval()

RecursiveScriptModule(
  original_name=NeuralNetwork
  (flatten): RecursiveScriptModule(original_name=Flatten)
  (linear_relu_stack): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=Linear)
  )
)

In [89]:
# meta: 提高资源利用率
with torch.device("meta"):
    model = NeuralNetwork()
print(next(model.parameters()))
model.load_state_dict(a,assign=True)
print(next(model.parameters()))


Parameter containing:
tensor(..., device='meta', size=(512, 784), requires_grad=True)
Parameter containing:
tensor([[ 0.0344, -0.0085,  0.0273,  ..., -0.0126, -0.0184, -0.0085],
        [ 0.0224, -0.0158, -0.0228,  ...,  0.0080,  0.0076, -0.0087],
        [ 0.0294, -0.0282, -0.0030,  ...,  0.0106,  0.0216,  0.0326],
        ...,
        [ 0.0043, -0.0345,  0.0024,  ...,  0.0143, -0.0183, -0.0164],
        [ 0.0248,  0.0318, -0.0303,  ..., -0.0027, -0.0020,  0.0215],
        [ 0.0274,  0.0347,  0.0193,  ..., -0.0213, -0.0307, -0.0321]],
       device='cuda:0', requires_grad=True)


In [ ]:

# 在运行推理之前，必须调用model.eval（）将退出层和批处理规范化层设置为inference模式。如果不这样做，将产生不一致的推理结果。

# model.state_dict（）返回对状态的引用，而不是它的副本！您必须序列化best_model_state或使用best_model_state = deepcopy(model.state_dict())，否则您的最佳best_model_state将在随后的训练迭代中不断更新。因此，最终的模型状态将是过拟合模型的状态。

# pickle模块保存整个模块。这种方法的缺点是序列化数据与保存模型时使用的特定类和确切的目录结构绑定在一起。原因是 pickle 不会保存模型类本身。相反，它会保存包含该类的文件的路径，该路径在加载时使用。因此，在其他项目中使用或重构后，您的代码可能会以各种方式中断。